In [ ]:
import os
import numpy as np
import deepspeech

# Set the path to the preprocessed data
data_dir = "/path/to/preprocessed-data"

# Set the path to the trained model
model_dir = "/path/to/trained-model"

# Set the hyperparameters for training the model
alphabet = deepspeech.Alphabet(os.path.join(deepspeech.Model.getDefaultModelPath(), "alphabet.txt"))
lm = deepspeech.LanguageModel(os.path.join(deepspeech.Model.getDefaultModelPath(), "lm.binary"))
trie = deepspeech.Trie(os.path.join(deepspeech.Model.getDefaultModelPath(), "trie"))
ds = deepspeech.Model(os.path.join(deepspeech.Model.getDefaultModelPath(), "deepspeech-0.9.3-models.pbmm"))
ds.enableExternalScorer(lm)
beam_width = 500
lm_alpha = 0.75
lm_beta = 1.85
n_features = 26
n_context = 9
batch_size = 32
n_epochs = 20
learning_rate = 0.0001

# Load the preprocessed data
X = []
y = []
for subdir in os.listdir(data_dir):
    subpath = os.path.join(data_dir, subdir)
    if not os.path.isdir(subpath):
        continue
    for filename in os.listdir(subpath):
        if not filename.endswith(".npy"):
            continue
        filepath = os.path.join(subpath, filename)
        # Load the preprocessed audio file
        X.append(np.load(filepath))
        # Load the corresponding transcription
        with open(os.path.join(subpath, filename[:-4] + ".txt")) as f:
            text = f.read().strip()
            y.append(text)

# Train the speech recognition model
for epoch in range(n_epochs):
    print("Epoch", epoch + 1)
    for i in range(0, len(X), batch_size):
        batch_X = X[i:i+batch_size]
        batch_y = y[i:i+batch_size]
        batch_len = [len(x) for x in batch_X]
        batch_X = [ds.stt(x, sampling_rate) for x in batch_X]
        ds.trainBatch(batch_X, batch_len, batch_y, learning_rate, beam_width, lm_alpha, lm_beta, alphabet, lm, trie, n_features, n_context)
    ds.save(os.path.join(model_dir, "epoch{}.pb".format(epoch + 1)))
